In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn import CrossEntropyLoss

In [2]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
    
from scipy import sparse
from os import path

In [3]:
#Baselines

In [65]:
softmax = nn.Softmax

In [4]:
import pickle

file_path = 'items_values_dict_Yahoo.pkl'

# Open the file in write binary mode and use pickle.dump to save the dictionary
#with open(file_path, 'wb') as f:
    #pickle.dump(items_values_dict, f)
with open(file_path, 'rb') as f:
    items_values_dict = pickle.load(f)    

In [5]:
import pickle

file_path = 'user_similarities_Jaccard_Yahoo.pkl'

# Open the file in write binary mode and use pickle.dump to save the dictionary
#with open(file_path, 'wb') as f:
    #pickle.dump(user_similarities_Jaccard, f)
with open(file_path, 'rb') as f:
    user_similarities_Jaccard = pickle.load(f)

In [6]:
import pickle

file_path = 'cosine_items_Yahoo.pkl'

# Open the file in write binary mode and use pickle.dump to save the dictionary
#with open(file_path, 'wb') as f:
    #pickle.dump(cosine_items_dict, f)
with open(file_path, 'rb') as f:
    cosine_items_dict = pickle.load(f)    

In [7]:
cosine_items = cosine_items_dict

In [8]:
import pickle

file_path = 'tf_idf_items_Yahoo.pkl'

# Open the file in write binary mode and use pickle.dump to save the dictionary
#with open(file_path, 'wb') as f:
    #pickle.dump(cosine_items_dict, f)
with open(file_path, 'rb') as f:
    tf_idf_items = pickle.load(f) 

In [9]:
import pickle

file_path = 'pop_dict_Yahoo.pkl'

with open(file_path, 'rb') as f:
    popularity_dict = pickle.load(f)  

In [10]:
##LIME

In [11]:
import importlib
import ipynb
from ipynb.fs.defs.lime import distance_to_proximity, LimeBase, get_lime_args, gaussian_kernel, mlp_get_lime_args
importlib.reload(ipynb.fs.defs.lime)
from ipynb.fs.defs.lime import distance_to_proximity, LimeBase, get_lime_args, gaussian_kernel, mlp_get_lime_args
'''
distance_to_proximity(distances_list) - takes distances from origin user and returns proximity
LimeBase() - class that gets kernel function
get_lime_args(user_vetor, item_id, model, items_array, min_pert = 10, max_pert = 20, num_of_perturbations = 5, seed = 0) - 
    returns neighborhood_data, neighborhood_labels, distances, item_id 
'''

lime = LimeBase(distance_to_proximity)

In [12]:
#Models

In [13]:
class MLP_G(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MLP_G, self).__init__()
        self.linear_x = nn.Linear(input_size, hidden_size, bias = False)
        self.linear_y = nn.Linear(input_size, hidden_size, bias = False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, user, item):
        user_representation = self.linear_x(user.float())
        item_representation = self.linear_y(item.float())
        dot_prod = torch.matmul(user_representation, item_representation.T)
        dot_sigmoid = self.sigmoid(dot_prod)
        
        return dot_sigmoid

In [14]:
class Recommender_G(nn.Module):
    def __init__(self, num_items, hidden_size):
        super(Recommender_G, self).__init__()
        self.mlp = MLP_G(num_items, hidden_size).to(device)

    def forward(self, user_vector, item_vector):
        user_vector = user_vector.to(device)
        item_vector = item_vector.to(device)
        output = self.mlp(user_vector, item_vector)
        return output.to(device)

In [15]:
class Explainer_G(nn.Module):
    def __init__(self, recommender_model_g, input_size, hidden_size):
        super(Explainer_G, self).__init__()
        
        backbone_children = list(recommender_model_g.children())[0]

        self.slice1 = nn.Sequential(*list(backbone_children.children())[:1])
        self.slice2 = nn.Sequential(*list(backbone_children.children())[1:2])
        self.bottleneck = nn.Sequential(
            nn.ReLU(),
            nn.Linear(in_features = hidden_size*2, out_features=input_size),
            nn.Sigmoid()
        ).to(device)

    def forward(self, user, item):
        slice1_output = self.slice1(user.float())
        slice2_output = self.slice2(item.float())
        combined_output = torch.cat((slice1_output, slice2_output), dim=-1)
        mask = self.bottleneck(combined_output).to(device)
        return mask

In [16]:

import torch.nn.functional as F
class LossModelCombined(torch.nn.Module):
    def __init__(self,alpha_parameter, recommender_model_g, explainer_model_g, hidden_size):
        
        super().__init__()
            
        self.recommender_model_g =  recommender_model_g
        self.explainer_model_g= explainer_model_g
        self.hidden_size = hidden_size
        self.alpha_parameter = alpha_parameter

        
    def forward(self,user_hist, y_true):
        
        user_hist = user_hist.to(device)
        y_true = torch.tensor(y_true).float().to(device)
        
        mask = self.explainer_model_g(user_hist,y_true).to(device)
         # "weakened" history 
        x_masked = user_hist * mask
        y_item_id = np.argmax(y_true.cpu().detach().numpy())
        
        y_masked = self.recommender_model_g(x_masked,y_true).unsqueeze(0).to(device)      
        #cross entropy between the new "masked" prediction and the original one
        cross_entropy_loss = -torch.log(y_masked).to(device)
        #l1 loss on mask
        l1_loss = torch.mean(torch.abs(mask)).to(device)
        #bce_loss = torch.nn.BCELoss(mask[y_item_id], y_true[y_item_id])#-torch.log(1-mask[y_item_id]).to(device)
        #combined loss
        comb_loss = cross_entropy_loss + self.alpha_parameter * l1_loss
        
        
        #print("Cross-entropy loss: {:.4f}, L1 loss: {:.4f}".format(cross_entropy_loss.item(), bce_loss.item()))
        
        return x_masked, comb_loss  
       

'\nimport torch.nn.functional as F\nclass LossModelCombined(torch.nn.Module):\n    def __init__(self,alpha_parameter, beta_parameter, recommender_model, explainer_model, hidden_size):\n     \n        super().__init__()\n            \n        self.recommender_model =  recommender_model\n        self.explainer_model= explainer_model\n        self.hidden_size = hidden_size\n        self.alpha_parameter = alpha_parameter\n        self.beta_parameter = beta_parameter\n        \n    def forward(self,user_hist, y_positive, y_negative=None):\n      \n        user_hist = user_hist.to(device)\n        y_positive = torch.tensor(y_positive).float().to(device)\n        \n        mask = self.explainer_model(user_hist,y_positive).to(device)\n        x_masked = user_hist * mask\n        y_positive_masked = self.recommender_model(x_masked,y_positive).unsqueeze(0).to(device)      \n        pred_loss = -torch.log(y_positive_masked).to(device)\n        \n        pred_negative_loss = 0\n        if y_negati

In [18]:
##Train and test Data

In [19]:
# Train the model on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"
hidden_dim = 20
num_users = 13725
num_items = 10265
print("num_users is ", num_users)
print("num_items is ", num_items)

num_users is  13725
num_items is  10265


In [54]:
rec_model = Recommender_G(num_items, hidden_dim)
rec_model.load_state_dict(torch.load('recommender_model_yahoo.pt'))

<All keys matched successfully>

In [55]:
for param in rec_model.parameters():
    param.requires_grad= False

In [22]:
##Read data from the files

In [23]:
train_data_mixed = pd.read_csv('train_data_mixed_Yahoo.csv')

In [24]:
test_data = pd.read_csv('test_data_Yahoo.csv')

In [25]:
train_data_mixed.head()

,0,1,2,3,4,5,6,7,8,9,...,10258,10259,10260,10261,10262,10263,10264,user_id,interaction,y_values
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7987,1,5303
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7987,0,3309
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,10191,1,7053
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,10191,0,2273
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3867,1,7389


In [26]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,10257,10258,10259,10260,10261,10262,10263,10264,user_id,y_positive
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4891,3025
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,12745,7426
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9355,5693
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6963,6269
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10601,6542


In [27]:
import pickle
with open('items_values_dict_Yahoo.pkl', 'rb') as f:
    items_values_dict = pickle.load(f) # deserialize using load()

In [28]:
items_values= pd.read_csv('items_values_Yahoo.csv')

In [29]:
items_array = items_values.to_numpy()

In [30]:
train_array = train_data_mixed.to_numpy()
test_array = test_data.to_numpy()

In [31]:
#Top k dictionaries for train and test

In [32]:
filename = 'topk_train_Yahoo.pkl'

# open the file in write-binary mode and save the array
with open(filename, 'rb') as f:
    topk_train = pickle.load(f)

In [33]:
filename = 'topk_test_Yahoo.pkl'

# open the file in write-binary mode and save the array
with open(filename, 'rb') as f:
    topk_test = pickle.load(f)

In [34]:
##Metrics calculation

In [35]:
def get_top_k(user_vector, original_user_vector, num_items, model, top_k):
    item_prob_dict = {}
    user_tensor = torch.Tensor(user_vector).to(device)
    item_tensor = torch.FloatTensor(items_array).to(device)
    output_model = [float(i) for i in model(user_tensor, item_tensor).cpu().detach().numpy()]
    
    original_user_vector = np.array(original_user_vector.cpu())
    neg = np.ones_like(original_user_vector)- original_user_vector
    output = neg*output_model
    for i in range(len(output)):
        item_prob_dict[i]=output[i]

    sorted_items_by_prob  = sorted(item_prob_dict.items(), key=lambda item: item[1],reverse=True)

    return dict(sorted_items_by_prob[0:top_k])

In [36]:
def get_index_in_the_list(user_vector, original_user_vector, item_id, num_items, model):
    top_k_list = list(get_top_k(user_vector, original_user_vector, num_items, model, num_items).keys())
    return top_k_list.index(item_id)

In [37]:
import warnings
warnings.filterwarnings('ignore')

In [38]:
def get_ndcg(ranked_list, target_item):
    if target_item not in ranked_list:
        return 0.0

    target_idx = torch.tensor(ranked_list.index(target_item), device=device)
    ndcg = torch.reciprocal(torch.log2(target_idx + 2))

    return ndcg.item()

In [69]:
def single_user_metrics(user_vector, item_id, k, num_of_bins, num_items, recommender_model, model_combined, mask_type = None):
    
    user_tensor = torch.FloatTensor(user_vector).to(device)
    
    item_vector = items_values_dict[item_id]
    item_tensor = torch.FloatTensor(item_vector).to(device)
    
    POS_masked = user_tensor
    NEG_masked = user_tensor
    POS_masked[item_id]=0
    NEG_masked[item_id]=0
    
    user_hist_size = np.sum(user_vector)

    
    bins=[0]+[len(x) for x in np.array_split(np.arange(np.sum(user_vector)), num_of_bins, axis=0)]
    
    POS_at_1 = [0]*(len(bins))
    POS_at_5 = [0]*(len(bins))
    POS_at_10=[0]*(len(bins))
    POS_at_20=[0]*(len(bins))
    POS_at_50=[0]*(len(bins))
    POS_at_100=[0]*(len(bins))
    
    NEG_at_1 = [0]*(len(bins))
    NEG_at_5 = [0]*(len(bins))
    NEG_at_10 = [0]*(len(bins))
    NEG_at_20 = [0]*(len(bins))
    NEG_at_50 = [0]*(len(bins))
    NEG_at_100 = [0]*(len(bins))
    
    DEL = [0]*(len(bins))
    INS = [0]*(len(bins))
    
    rankA_at_1 = [0]*(len(bins))
    rankA_at_5 = [0]*(len(bins))
    rankA_at_10 = [0]*(len(bins))
    rankA_at_20 = [0]*(len(bins))
    rankA_at_50 = [0]*(len(bins))
    rankA_at_100 = [0]*(len(bins))
    
    rankB = [0]*(len(bins))
    
    NDCG_at_1 = [0]*(len(bins))
    NDCG_at_5 = [0]*(len(bins))
    NDCG_at_10 = [0]*(len(bins))
    NDCG_at_20 = [0]*(len(bins))
    NDCG_at_50 = [0]*(len(bins))
    NDCG_at_100 = [0]*(len(bins))
    
    total_items = 0
    
    for i in range(len(bins)):
        total_items += bins[i]
        
        if i == 0:
            if mask_type == 'jaccard_g':
                sim_items = find_jaccard_g_mask(POS_masked, item_id, num_items, jaccard_based_sim)
            elif mask_type == 'jaccard_u':
                sim_items = find_jaccard_u_mask(POS_masked, item_id, num_items, user_similarities_Jaccard)
            elif mask_type == 'cosine':
                sim_items = find_cosine_mask(POS_masked, item_id, num_items, cosine_items)
            elif mask_type == 'pop':
                sim_items = find_pop_mask(POS_masked, item_id, num_items)
            elif mask_type == 'tf_idf':
                sim_items = find_tf_idf_mask(POS_masked, item_id, num_items, tf_idf_items)
            elif mask_type == 'lime':
                sim_items = find_LIME_mask(user_vector, item_id, items_array, 50, 100, 50, distance_to_proximity,'highest_weights',recommender_model, num_samples=user_hist_size)
            elif mask_type == 'ltx':
                sim_items = find_LTX_mask(POS_masked, item_vector, item_id, model_combined)   
            else:
                raise Exception("Wrong mask type")
        
        POS_sim_items  = list(sorted(sim_items.items(), key=lambda item: item[1],reverse=True))[0:user_hist_size]
        NEG_sim_items  = list(sorted(dict(POS_sim_items).items(), key=lambda item: item[1],reverse=False))[0:user_hist_size]
        
        POS_masked = torch.zeros_like(user_tensor, dtype=torch.float32, device=device)
        for j in POS_sim_items[:total_items]:
            POS_masked[j[0]] = 1
        POS_masked = user_tensor - POS_masked
        
        NEG_masked = torch.zeros_like(user_tensor, dtype=torch.float32, device=device)
        for j in NEG_sim_items[:total_items]:
            NEG_masked[j[0]] = 1
        NEG_masked = user_tensor - NEG_masked # remove the masked items from the user history 

        POS_ranked_list = get_top_k(POS_masked, user_tensor,num_items, recommender_model,num_items)        
        POS_index = list(POS_ranked_list.keys()).index(item_id)+1
        NEG_index = get_index_in_the_list(NEG_masked,user_tensor, item_id, num_items, recommender_model)+1

        # for pos:
        POS_at_1[i] = 1 if POS_index <=1 else 0
        POS_at_5[i] = 1 if POS_index <=5 else 0
        POS_at_10[i] = 1 if POS_index <=10 else 0
        POS_at_20[i] = 1 if POS_index <=20 else 0
        POS_at_50[i] = 1 if POS_index <=50 else 0
        POS_at_100[i] = 1 if POS_index <=100 else 0

        # for neg:
        NEG_at_1[i] = 1 if NEG_index <=1 else 0
        NEG_at_5[i] = 1 if NEG_index <=5 else 0
        NEG_at_10[i] = 1 if NEG_index <=10 else 0
        NEG_at_20[i] = 1 if NEG_index <=20 else 0
        NEG_at_50[i] = 1 if NEG_index <=50 else 0
        NEG_at_100[i] = 1 if NEG_index <=100 else 0

        # for del:
        DEL[i] = float(recommender_model(POS_masked, item_tensor).detach().cpu().numpy())
        
        # for ins:
        INS[i] = float(recommender_model(user_tensor-POS_masked, item_tensor).detach().cpu().numpy())
        

        # for rankA:
        rankA_at_1[i] = max(0, (1+1-POS_index)/10)
        rankA_at_5[i] = max(0, (5+1-POS_index)/20)
        rankA_at_10[i] = max(0, (10+1-POS_index)/10)
        rankA_at_20[i] = max(0, (20+1-POS_index)/20)
        rankA_at_50[i] = max(0, (50+1-POS_index)/50)
        rankA_at_100[i] = max(0, (100+1-POS_index)/100)

        # for rankB:
        rankB[i] = 1/POS_index

        #for NDCG:
        NDCG_at_1[i]= get_ndcg(list(POS_ranked_list.keys())[:1],item_id)
        NDCG_at_5[i]= get_ndcg(list(POS_ranked_list.keys())[:5],item_id)
        NDCG_at_10[i]= get_ndcg(list(POS_ranked_list.keys())[:10],item_id)
        NDCG_at_20[i]= get_ndcg(list(POS_ranked_list.keys())[:20],item_id)
        NDCG_at_50[i]= get_ndcg(list(POS_ranked_list.keys())[:50],item_id)
        NDCG_at_100[i]= get_ndcg(list(POS_ranked_list.keys())[:100],item_id)

    res = [POS_at_1, POS_at_5, POS_at_10, POS_at_20, POS_at_50, POS_at_100, NEG_at_1, NEG_at_5, NEG_at_10, NEG_at_20, NEG_at_50, NEG_at_100, DEL, INS, rankA_at_1, rankA_at_5, rankA_at_10, rankA_at_20, rankA_at_50, rankA_at_100, rankB, NDCG_at_1, NDCG_at_5, NDCG_at_10, NDCG_at_20, NDCG_at_50, NDCG_at_100]
    for i in range(len(res)):
        res[i] = np.array(res[i])
        
    return res

In [37]:
'''
def single_user_metrics(user_vector, item_id, k, num_of_bins, num_items, recommender_model, model_combined, mask_type = None):
    
    user_tensor = torch.FloatTensor(user_vector).to(device)
    
    item_vector = items_values_dict[item_id]
    item_tensor = torch.FloatTensor(item_vector).to(device)
    
    POS_masked = user_tensor
    NEG_masked = user_tensor
    POS_masked[item_id]=0
    NEG_masked[item_id]=0
    
    user_hist_size = np.sum(user_vector)

    
    bins=[0]+[len(x) for x in np.array_split(np.arange(np.sum(user_vector)), num_of_bins, axis=0)]
    
    POS_at_10=[0]*(len(bins))
    POS_at_20=[0]*(len(bins))
    POS_at_50=[0]*(len(bins))
    POS_at_100=[0]*(len(bins))
    NEG_at_10 = [0]*(len(bins))
    NEG_at_20 = [0]*(len(bins))
    NEG_at_50 = [0]*(len(bins))
    NEG_at_100 = [0]*(len(bins))
    DEL = [0]*(len(bins))
    INS = [0]*(len(bins))
    rankA_at_10 = [0]*(len(bins))
    rankA_at_20 = [0]*(len(bins))
    rankA_at_50 = [0]*(len(bins))
    rankA_at_100 = [0]*(len(bins))
    rankB = [0]*(len(bins))
    NDCG_at_10 = [0]*(len(bins))
    NDCG_at_20 = [0]*(len(bins))
    NDCG_at_50 = [0]*(len(bins))
    NDCG_at_100 = [0]*(len(bins))
    
    total_items = 0
    
    for i in range(len(bins)):
        total_items += bins[i]
        
        if i == 0:
            if mask_type == 'jaccard_g':
                sim_items = find_jaccard_g_mask(POS_masked, item_id, num_items, jaccard_based_sim)
            elif mask_type == 'jaccard_u':
                sim_items = find_jaccard_u_mask(POS_masked, item_id, num_items, user_similarities_Jaccard)
            elif mask_type == 'cosine':
                sim_items = find_cosine_mask(POS_masked, item_id, num_items, cosine_items)
            elif mask_type == 'tf_idf':
                sim_items = find_tf_idf_mask(POS_masked, item_id, num_items, tf_idf_items)
            elif mask_type == 'pop':
                sim_items = find_pop_mask(POS_masked, item_id, num_items)
            elif mask_type == 'lime':
                sim_items = find_LIME_mask(user_vector, item_id, items_array, 50, 100, 400, distance_to_proximity,'highest_weights',recommender_model, num_samples=user_hist_size)
            elif mask_type == 'ltx':
                sim_items = find_LTX_mask(POS_masked, item_vector, item_id, model_combined)   
            else:
                raise Exception("Wrong mask type")
        
        POS_sim_items  = list(sorted(sim_items.items(), key=lambda item: item[1],reverse=True))[0:user_hist_size]
        NEG_sim_items  = list(sorted(dict(POS_sim_items).items(), key=lambda item: item[1],reverse=False))
        
        
        NEG_masked = torch.zeros_like(user_tensor, dtype=torch.float32, device=device)
        for j in NEG_sim_items[:total_items]:
            NEG_masked[j[0]] = 1
        NEG_masked = user_tensor - NEG_masked # remove the masked items from the user history 


        NEG_index = get_index_in_the_list(NEG_masked,user_tensor, item_id, num_items, recommender_model)+1
   
        
        # for neg:
        NEG_at_10[i] = 1 if NEG_index <=10 else 0
        NEG_at_20[i] = 1 if NEG_index <=20 else 0
        NEG_at_50[i] = 1 if NEG_index <=50 else 0
        NEG_at_100[i] = 1 if NEG_index <=100 else 0
        
 
            
    res = [POS_at_10, POS_at_20, POS_at_50, POS_at_100, NEG_at_10, NEG_at_20, NEG_at_50, NEG_at_100, DEL, INS, rankA_at_10, rankA_at_20, rankA_at_50, rankA_at_100, rankB, NDCG_at_10, NDCG_at_20, NDCG_at_50, NDCG_at_100]
    for i in range(len(res)):
        res[i] = np.array(res[i])
    return res  
    '''

In [38]:
#Mask calculation

In [41]:
def find_LIME_mask(x, item_id, items_array, min_pert, max_pert, num_of_perturbations, kernel_func, feature_selection, model, num_samples=10, method = 'POS'):
    
    user_hist = x 
    # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    lime.kernel_fn = kernel_func
    neighborhood_data, neighborhood_labels, distances, item_id = mlp_get_lime_args(user_hist, item_id, model, items_array, min_pert = min_pert, max_pert = max_pert, num_of_perturbations = num_of_perturbations, seed = item_id)
                                                                                  
    most_pop_items  = lime.explain_instance_with_data(neighborhood_data, neighborhood_labels, distances, item_id, num_samples, feature_selection, pos_neg='POS')
    most_pop_items_dict =  {key: value for key, value in most_pop_items}
    
    return most_pop_items_dict

In [42]:
#LTX based similarity
def find_LTX_mask(x, y_true, item_id, model_combined):
    
    user_hist = torch.tensor(x) 
    user_hist[item_id] = 0 
    
    x_masked_g, loss_comb_g = model_combined(user_hist, y_true)
    
    x_masked_g = x_masked_g.to(device)
    #item_sim_dict = {i: v for i, v in enumerate(x_masked_l)}
    item_sim_dict = {i: x_masked_g[i].item() for i in range(x_masked_g.numel())}
    
    return item_sim_dict

In [43]:
#Genre based similarities using Jaccard
def find_jaccard_g_mask(x, item_id, num_items,jaccard_based_sim):
    user_hist = torch.tensor(x) # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    item_jaccard_dict = {}
    for i in range(num_items): 
        if(user_hist[i] == 1): # iterate all positive items of the user
            if((i,item_id) in jaccard_based_sim.keys()):
                item_jaccard_dict[i]=jaccard_based_sim[(i,item_id)] # add Jaccard similarity between items
            else:
                item_jaccard_dict[i] = 0
    
    return item_jaccard_dict

In [44]:
#User based similarities using Jaccard
def find_jaccard_u_mask(x, item_id, num_items, user_based_Jaccard_sim):
    user_hist = torch.tensor(x) # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    item_jaccard_dict = {}
    for i in range(num_items): 
        if(user_hist[i] == 1): # iterate all positive items of the user
            if((i,item_id) in user_based_Jaccard_sim.keys()):
                item_jaccard_dict[i]=user_based_Jaccard_sim[(i,item_id)] # add Jaccard similarity between items
            else:
                item_jaccard_dict[i] = 0
            
    return item_jaccard_dict

In [45]:
#Cosine based similarities between users and items
def find_cosine_mask(x, item_id, num_items, item_cosine):

    user_hist = torch.tensor(x) # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    item_cosine_dict = {}
        
    for i in range(num_items): 
        if(user_hist[i] == 1): # iterate all positive items of the user
            if((i,item_id) in item_cosine.keys()):
                item_cosine_dict[i]=item_cosine[(i,item_id)] # add cosine similarity between items
            else:
                item_cosine_dict[i] = 0
    
    return item_cosine_dict

In [46]:
#tf_idf mask
def find_tf_idf_mask(x, item_id, num_items, tf_idf_sim):
    
    x = x.cpu().detach().numpy()
    x[item_id] = 0
    
    positive_items = np.where(x == 1)[0]
    tf_idf_dict = {i: tf_idf_sim.get((i, item_id), 0) for i in positive_items}
    
    return tf_idf_dict

In [47]:
#popularity mask
def find_pop_mask(x, item_id, num_items):
    user_hist = torch.tensor(x) # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    item_pop_dict = {}
    for i in range(num_items): 
        if(user_hist[i] == 1): # iterate over all positive items of the user
            item_pop_dict[i]=popularity_dict[i] # add the pop of the item to the dictionary
            
            
    return item_pop_dict

In [48]:
#LTX train

In [49]:
#Get users vectors to create topk
unique_indices = np.unique(train_array[:,-3], return_index=True, axis=0)[1]

# create a new array with only the unique users
train_unique_arr = train_array[unique_indices, :]

In [56]:
explainer_model_g = Explainer_G(rec_model, num_items, hidden_dim).to(device) 

In [57]:
#LTX training

import time
import random
train_losses = []
epochs = 13
#torch.manual_seed(42)
hidden_dim = 20
num_of_bins = 10
alpha_parameter = 0.1
#beta_parameter = 0.1

model_combined = LossModelCombined(alpha_parameter,rec_model, explainer_model_g, hidden_dim).to(device)
optimizer_comb = torch.optim.Adam(model_combined.parameters(), lr=0.01)
    
#Train LTX
for epoch in range(epochs):
    train_loss = 0
    
    for i in range(train_unique_arr.shape[0]):
        
        #user data
        user_id = train_unique_arr[i][-3]
        user_vector = train_unique_arr[i][:-3]
        #get top1 of this user for LTX training
        
        top1_item = np.argmax(topk_train[user_id])
    
        #top_indices = np.argsort(list(topk_train[user_id]))[-100:-1]
        # Sample one index randomly
        #top_100_indices = np.delete(top_indices, np.where(top_indices == top1_item))
    
        # sample an index from the top 100
        #not_top1 = np.random.choice(top_100_indices)
        #print(not_top1)
        #print("not_top1_item ", not_top1)
       
        positive_sample = torch.FloatTensor(items_values_dict[top1_item]).to(device)
        #negative_sample = torch.FloatTensor(items_values_dict[not_top1]).to(device)
        
        user_vector[top1_item] = 0 
        user_tensor = torch.FloatTensor(user_vector).to(device)

        #train model        
        optimizer_comb.zero_grad()
        x_masked_g, loss_comb_g = model_combined(user_tensor, positive_sample) #, negative_sample)
        
        train_loss+=loss_comb_g.item()
            
        loss_comb_g.backward()
        optimizer_comb.step()
        
  
    train_losses.append(train_loss/train_unique_arr.shape[0])

    print(f"Epoch {epoch}, Train Loss {train_loss/train_unique_arr.shape[0]:.4f}")

Epoch 0, Train Loss 0.0467
Epoch 1, Train Loss 0.0149
Epoch 2, Train Loss 0.0087
Epoch 3, Train Loss 0.0063
Epoch 4, Train Loss 0.0058
Epoch 5, Train Loss 0.0055
Epoch 6, Train Loss 0.0047
Epoch 7, Train Loss 0.0035
Epoch 8, Train Loss 0.0038
Epoch 9, Train Loss 0.0030
Epoch 10, Train Loss 0.0034
Epoch 11, Train Loss 0.0031
Epoch 12, Train Loss 0.0031


In [58]:
torch.save(model_combined.state_dict(), 'model_combined_yahoo_13_01.pt')

In [59]:
torch.save(explainer_model_g.state_dict(), 'explainer_model_yahoo_13_01.pt')

In [52]:
#LTX on testing data set

In [ ]:
import time
# Evaluate the model on the test set
num_of_bins = 11
k = 100



POS_at_1_j_g = np.zeros(num_of_bins)
pos_at_5_j_g = np.zeros(num_of_bins)
POS_at_10_j_g = np.zeros(num_of_bins)
POS_at_20_j_g = np.zeros(num_of_bins)
POS_at_50_j_g = np.zeros(num_of_bins)
POS_at_100_j_g = np.zeros(num_of_bins)
NEG_at_1_j_g = np.zeros(num_of_bins)
NEG_at_5_j_g = np.zeros(num_of_bins)
NEG_at_10_j_g = np.zeros(num_of_bins)
NEG_at_20_j_g = np.zeros(num_of_bins)
NEG_at_50_j_g = np.zeros(num_of_bins)
NEG_at_100_j_g = np.zeros(num_of_bins)
users_DEL_j_g = np.zeros(num_of_bins)
users_INS_j_g = np.zeros(num_of_bins)
rank_at_1_j_g = np.zeros(num_of_bins)
rank_at_5_j_g = np.zeros(num_of_bins)
rank_at_10_A_j_g = np.zeros(num_of_bins)
rank_at_20_A_j_g = np.zeros(num_of_bins)
rank_at_50_A_j_g = np.zeros(num_of_bins)
rank_at_100_A_j_g = np.zeros(num_of_bins)
rank_at_k_B_j_g = np.zeros(num_of_bins)
NDCG_at_1_j_g = np.zeros(num_of_bins)
NDCG_at_5_j_g = np.zeros(num_of_bins)
NDCG_at_10_j_g = np.zeros(num_of_bins)
NDCG_at_20_j_g = np.zeros(num_of_bins)
NDCG_at_50_j_g = np.zeros(num_of_bins)
NDCG_at_100_j_g = np.zeros(num_of_bins)


POS_at_1_j_u = np.zeros(num_of_bins)
pos_at_5_j_u = np.zeros(num_of_bins)
POS_at_10_j_u = np.zeros(num_of_bins)
POS_at_20_j_u = np.zeros(num_of_bins)
POS_at_50_j_u = np.zeros(num_of_bins)
POS_at_100_j_u = np.zeros(num_of_bins)
NEG_at_1_j_u = np.zeros(num_of_bins)
NEG_at_5_j_u = np.zeros(num_of_bins)
NEG_at_10_j_u = np.zeros(num_of_bins)
NEG_at_20_j_u = np.zeros(num_of_bins)
NEG_at_50_j_u = np.zeros(num_of_bins)
NEG_at_100_j_u = np.zeros(num_of_bins)
users_DEL_j_u = np.zeros(num_of_bins)
users_INS_j_u = np.zeros(num_of_bins)
rank_at_1_j_u = np.zeros(num_of_bins)
rank_at_5_j_u = np.zeros(num_of_bins)
rank_at_10_A_j_u = np.zeros(num_of_bins)
rank_at_20_A_j_u = np.zeros(num_of_bins)
rank_at_50_A_j_u = np.zeros(num_of_bins)
rank_at_100_A_j_u = np.zeros(num_of_bins)
rank_at_k_B_j_u = np.zeros(num_of_bins)
NDCG_at_1_j_u = np.zeros(num_of_bins)
NDCG_at_5_j_u = np.zeros(num_of_bins)
NDCG_at_10_j_u = np.zeros(num_of_bins)
NDCG_at_20_j_u = np.zeros(num_of_bins)
NDCG_at_50_j_u = np.zeros(num_of_bins)
NDCG_at_100_j_u = np.zeros(num_of_bins)


POS_at_1_c_s = np.zeros(num_of_bins)
pos_at_5_c_s = np.zeros(num_of_bins)
POS_at_10_c_s = np.zeros(num_of_bins)
POS_at_20_c_s = np.zeros(num_of_bins)
POS_at_50_c_s = np.zeros(num_of_bins)
POS_at_100_c_s = np.zeros(num_of_bins)
NEG_at_1_c_s = np.zeros(num_of_bins)
NEG_at_5_c_s = np.zeros(num_of_bins)
NEG_at_10_c_s = np.zeros(num_of_bins)
NEG_at_20_c_s= np.zeros(num_of_bins)
NEG_at_50_c_s = np.zeros(num_of_bins)
NEG_at_100_c_s = np.zeros(num_of_bins)
users_DEL_c_s = np.zeros(num_of_bins)
users_INS_c_s = np.zeros(num_of_bins)
rank_at_1_c_s = np.zeros(num_of_bins)
rank_at_5_c_s = np.zeros(num_of_bins)
rank_at_10_A_c_s = np.zeros(num_of_bins)
rank_at_20_A_c_s = np.zeros(num_of_bins)
rank_at_50_A_c_s = np.zeros(num_of_bins)
rank_at_100_A_c_s = np.zeros(num_of_bins)
rank_at_k_B_c_s = np.zeros(num_of_bins)
NDCG_at_1_c_s = np.zeros(num_of_bins)
NDCG_at_5_c_s = np.zeros(num_of_bins)
NDCG_at_10_c_s = np.zeros(num_of_bins)
NDCG_at_20_c_s = np.zeros(num_of_bins)
NDCG_at_50_c_s = np.zeros(num_of_bins)
NDCG_at_100_c_s = np.zeros(num_of_bins)


POS_at_1_pop = np.zeros(num_of_bins)
pos_at_5_pop = np.zeros(num_of_bins)
POS_at_10_pop = np.zeros(num_of_bins)
POS_at_20_pop = np.zeros(num_of_bins)
POS_at_50_pop = np.zeros(num_of_bins)
POS_at_100_pop = np.zeros(num_of_bins)
NEG_at_1_pop = np.zeros(num_of_bins)
NEG_at_5_pop = np.zeros(num_of_bins)
NEG_at_10_pop = np.zeros(num_of_bins)
NEG_at_20_pop = np.zeros(num_of_bins)
NEG_at_50_pop = np.zeros(num_of_bins)
NEG_at_100_pop = np.zeros(num_of_bins)
users_DEL_pop = np.zeros(num_of_bins)
users_INS_pop = np.zeros(num_of_bins)
rank_at_1_pop = np.zeros(num_of_bins)
rank_at_5_pop = np.zeros(num_of_bins)
rank_at_10_A_pop = np.zeros(num_of_bins)
rank_at_20_A_pop = np.zeros(num_of_bins)
rank_at_50_A_pop = np.zeros(num_of_bins)
rank_at_100_A_pop = np.zeros(num_of_bins)
rank_at_k_B_pop = np.zeros(num_of_bins)
NDCG_at_1_pop = np.zeros(num_of_bins)
NDCG_at_5_pop = np.zeros(num_of_bins)
NDCG_at_10_pop = np.zeros(num_of_bins)
NDCG_at_20_pop = np.zeros(num_of_bins)
NDCG_at_50_pop = np.zeros(num_of_bins)
NDCG_at_100_pop = np.zeros(num_of_bins)

POS_at_1_lime = np.zeros(num_of_bins)
pos_at_5_lime = np.zeros(num_of_bins)
POS_at_10_lime = np.zeros(num_of_bins)
POS_at_20_lime = np.zeros(num_of_bins)
POS_at_50_lime = np.zeros(num_of_bins)
POS_at_100_lime = np.zeros(num_of_bins)
NEG_at_1_lime = np.zeros(num_of_bins)
NEG_at_5_lime = np.zeros(num_of_bins)
NEG_at_10_lime = np.zeros(num_of_bins)
NEG_at_20_lime = np.zeros(num_of_bins)
NEG_at_50_lime = np.zeros(num_of_bins)
NEG_at_100_lime = np.zeros(num_of_bins)
users_DEL_lime = np.zeros(num_of_bins)
users_INS_lime = np.zeros(num_of_bins)
rank_at_1_lime = np.zeros(num_of_bins)
rank_at_5_lime = np.zeros(num_of_bins)
rank_at_10_A_lime = np.zeros(num_of_bins)
rank_at_20_A_lime = np.zeros(num_of_bins)
rank_at_50_A_lime = np.zeros(num_of_bins)
rank_at_100_A_lime = np.zeros(num_of_bins)
rank_at_k_B_lime = np.zeros(num_of_bins)
NDCG_at_1_lime = np.zeros(num_of_bins)
NDCG_at_5_lime = np.zeros(num_of_bins)
NDCG_at_10_lime = np.zeros(num_of_bins)
NDCG_at_20_lime = np.zeros(num_of_bins)
NDCG_at_50_lime = np.zeros(num_of_bins)
NDCG_at_100_lime = np.zeros(num_of_bins)

POS_at_1_tf_idf = np.zeros(num_of_bins)
pos_at_5_tf_idf = np.zeros(num_of_bins)
POS_at_10_tf_idf = np.zeros(num_of_bins)
POS_at_20_tf_idf = np.zeros(num_of_bins)
POS_at_50_tf_idf = np.zeros(num_of_bins)
POS_at_100_tf_idf = np.zeros(num_of_bins)
NEG_at_1_tf_idf = np.zeros(num_of_bins)
NEG_at_5_tf_idf = np.zeros(num_of_bins)
NEG_at_10_tf_idf = np.zeros(num_of_bins)
NEG_at_20_tf_idf = np.zeros(num_of_bins)
NEG_at_50_tf_idf = np.zeros(num_of_bins)
NEG_at_100_tf_idf = np.zeros(num_of_bins)
users_DEL_tf_idf = np.zeros(num_of_bins)
users_INS_tf_idf = np.zeros(num_of_bins)
rank_at_1_tf_idf = np.zeros(num_of_bins)
rank_at_5_tf_idf = np.zeros(num_of_bins)
rank_at_10_A_tf_idf = np.zeros(num_of_bins)
rank_at_20_A_tf_idf = np.zeros(num_of_bins)
rank_at_50_A_tf_idf = np.zeros(num_of_bins)
rank_at_100_A_tf_idf = np.zeros(num_of_bins)
rank_at_k_B_tf_idf = np.zeros(num_of_bins)
NDCG_at_1_tf_idf = np.zeros(num_of_bins)
NDCG_at_5_tf_idf = np.zeros(num_of_bins)
NDCG_at_10_tf_idf = np.zeros(num_of_bins)
NDCG_at_20_tf_idf = np.zeros(num_of_bins)
NDCG_at_50_tf_idf = np.zeros(num_of_bins)
NDCG_at_100_tf_idf = np.zeros(num_of_bins)

POS_at_1_ltx = np.zeros(num_of_bins)
pos_at_5_ltx = np.zeros(num_of_bins)
POS_at_10_ltx = np.zeros(num_of_bins)
POS_at_20_ltx = np.zeros(num_of_bins)
POS_at_50_ltx = np.zeros(num_of_bins)
POS_at_100_ltx = np.zeros(num_of_bins)
NEG_at_1_ltx = np.zeros(num_of_bins)
NEG_at_5_ltx = np.zeros(num_of_bins)
NEG_at_10_ltx = np.zeros(num_of_bins)
NEG_at_20_ltx = np.zeros(num_of_bins)
NEG_at_50_ltx = np.zeros(num_of_bins)
NEG_at_100_ltx = np.zeros(num_of_bins)
users_DEL_ltx = np.zeros(num_of_bins)
users_INS_ltx = np.zeros(num_of_bins)
rank_at_1_ltx = np.zeros(num_of_bins)
rank_at_5_ltx = np.zeros(num_of_bins)
rank_at_10_A_ltx = np.zeros(num_of_bins)
rank_at_20_A_ltx = np.zeros(num_of_bins)
rank_at_50_A_ltx = np.zeros(num_of_bins)
rank_at_100_A_ltx = np.zeros(num_of_bins)
rank_at_k_B_ltx = np.zeros(num_of_bins)
NDCG_at_1_ltx = np.zeros(num_of_bins)
NDCG_at_5_ltx = np.zeros(num_of_bins)
NDCG_at_10_ltx = np.zeros(num_of_bins)
NDCG_at_20_ltx = np.zeros(num_of_bins)
NDCG_at_50_ltx = np.zeros(num_of_bins)
NDCG_at_100_ltx = np.zeros(num_of_bins)

num_of_bins = 10

explainer_model_g.eval()
model_combined.eval()

with torch.no_grad():
    for i in range(test_array.shape[0]):

        if i%500 == 0:
            print(i)
        #item_id = test_array[i][-1]
        user_id = test_array[i][-2]

        #item_vector = items_values.iloc[item_id]
        user_vector = test_array[i][:-2]

        y_pred = topk_test[user_id]
        item_id = np.argmax(y_pred)

        item_vector = items_values_dict[item_id]
        item_tensor = torch.FloatTensor(item_vector).to(device)

        user_vector[item_id] = 0
        user_tensor = torch.FloatTensor(user_vector).to(device)

        ### Jaccard user:
        res = single_user_metrics(user_vector, item_id, k, num_of_bins, num_items, rec_model, model_combined, mask_type= 'jaccard_u')
        POS_at_1_j_u += res[0]
        pos_at_5_j_u += res[1]
        POS_at_10_j_u += res[2]
        POS_at_20_j_u += res[3]
        POS_at_50_j_u += res[4]
        POS_at_100_j_u += res[5]
        NEG_at_1_j_u += res[6]
        NEG_at_5_j_u += res[7]
        NEG_at_10_j_u += res[8]
        NEG_at_20_j_u += res[9]
        NEG_at_50_j_u += res[10]
        NEG_at_100_j_u += res[11]
        users_DEL_j_u += res[12]
        users_INS_j_u += res[13]
        rank_at_1_j_u += res[14]
        rank_at_5_j_u += res[15]
        rank_at_10_A_j_u += res[16]
        rank_at_20_A_j_u += res[17]
        rank_at_50_A_j_u += res[18]
        rank_at_100_A_j_u += res[19]
        rank_at_k_B_j_u += res[20]
        NDCG_at_1_j_u += res[21]
        NDCG_at_5_j_u += res[22]
        NDCG_at_10_j_u += res[23]
        NDCG_at_20_j_u += res[24]
        NDCG_at_50_j_u += res[25]
        NDCG_at_100_j_u += res[26]

        ### cosine similarity:
        res = single_user_metrics(user_vector, item_id, k, num_of_bins, num_items, rec_model, model_combined, mask_type= 'cosine')
        POS_at_1_c_s += res[0]
        pos_at_5_c_s += res[1]
        POS_at_10_c_s += res[2]
        POS_at_20_c_s += res[3]
        POS_at_50_c_s += res[4]
        POS_at_100_c_s += res[5]
        NEG_at_1_c_s += res[6]
        NEG_at_5_c_s += res[7]
        NEG_at_10_c_s += res[8]
        NEG_at_20_c_s += res[9]
        NEG_at_50_c_s += res[10]
        NEG_at_100_c_s += res[11]
        users_DEL_c_s += res[12]
        users_INS_c_s += res[13]
        rank_at_1_c_s += res[14]
        rank_at_5_c_s += res[15]
        rank_at_10_A_c_s += res[16]
        rank_at_20_A_c_s += res[17]
        rank_at_50_A_c_s += res[18]
        rank_at_100_A_c_s += res[19]
        rank_at_k_B_c_s += res[20]
        NDCG_at_1_c_s += res[21]
        NDCG_at_5_c_s += res[22]
        NDCG_at_10_c_s += res[23]
        NDCG_at_20_c_s += res[24]
        NDCG_at_50_c_s += res[25]
        NDCG_at_100_c_s += res[26]
        
        ## tf-idf similarity:
        res = single_user_metrics(user_vector, item_id, k, num_of_bins, num_items, rec_model, model_combined, mask_type= 'tf_idf')
        POS_at_1_tf_idf += res[0]
        pos_at_5_tf_idf += res[1]
        POS_at_10_tf_idf += res[2]
        POS_at_20_tf_idf += res[3]
        POS_at_50_tf_idf += res[4]
        POS_at_100_tf_idf += res[5]
        NEG_at_1_tf_idf += res[6]
        NEG_at_5_tf_idf += res[7]
        NEG_at_10_tf_idf += res[8]
        NEG_at_20_tf_idf += res[9]
        NEG_at_50_tf_idf += res[10]
        NEG_at_100_tf_idf += res[11]
        users_DEL_tf_idf += res[12]
        users_INS_tf_idf += res[13]
        rank_at_1_tf_idf += res[14]
        rank_at_5_tf_idf += res[15]
        rank_at_10_A_tf_idf += res[16]
        rank_at_20_A_tf_idf += res[17]
        rank_at_50_A_tf_idf += res[18]
        rank_at_100_A_tf_idf += res[19]
        rank_at_k_B_tf_idf += res[20]
        NDCG_at_1_tf_idf += res[21]
        NDCG_at_5_tf_idf += res[22]
        NDCG_at_10_tf_idf += res[23]
        NDCG_at_20_tf_idf += res[24]
        NDCG_at_50_tf_idf += res[25]
        NDCG_at_100_tf_idf += res[26]

        ### pop:
        res = single_user_metrics(user_vector, item_id, k, num_of_bins, num_items, rec_model, model_combined, mask_type= 'pop')    
        POS_at_1_pop += res[0]
        pos_at_5_pop += res[1]
        POS_at_10_pop += res[2]
        POS_at_20_pop += res[3]
        POS_at_50_pop += res[4]
        POS_at_100_pop += res[5]
        NEG_at_1_pop += res[6]
        NEG_at_5_pop += res[7]
        NEG_at_10_pop += res[8]
        NEG_at_20_pop += res[9]
        NEG_at_50_pop += res[10]
        NEG_at_100_pop += res[11]
        users_DEL_pop += res[12]
        users_INS_pop += res[13]
        rank_at_1_pop += res[14]
        rank_at_5_pop += res[15]
        rank_at_10_A_pop += res[16]
        rank_at_20_A_pop += res[17]
        rank_at_50_A_pop += res[18]
        rank_at_100_A_pop += res[19]
        rank_at_k_B_pop += res[20]
        NDCG_at_1_pop += res[21]
        NDCG_at_5_pop += res[22]
        NDCG_at_10_pop += res[23]
        NDCG_at_20_pop += res[24]
        NDCG_at_50_pop += res[25]
        NDCG_at_100_pop += res[26]

        ### LTX:
        res = single_user_metrics(user_vector, item_id, k, num_of_bins, num_items, rec_model, model_combined, mask_type= 'ltx')    
        POS_at_1_ltx += res[0]
        pos_at_5_ltx += res[1]
        POS_at_10_ltx += res[2]
        POS_at_20_ltx += res[3]
        POS_at_50_ltx += res[4]
        POS_at_100_ltx += res[5]
        NEG_at_1_ltx += res[6]
        NEG_at_5_ltx += res[7]
        NEG_at_10_ltx += res[8]
        NEG_at_20_ltx += res[9]
        NEG_at_50_ltx += res[10]
        NEG_at_100_ltx += res[11]
        users_DEL_ltx += res[12]
        users_INS_ltx += res[13]
        rank_at_1_ltx += res[14]
        rank_at_5_ltx += res[15]
        rank_at_10_A_ltx += res[16]
        rank_at_20_A_ltx += res[17]
        rank_at_50_A_ltx += res[18]
        rank_at_100_A_ltx += res[19]
        rank_at_k_B_ltx += res[20]
        NDCG_at_1_ltx += res[21]
        NDCG_at_5_ltx += res[22]
        NDCG_at_10_ltx += res[23]
        NDCG_at_20_ltx += res[24]
        NDCG_at_50_ltx += res[25]
        NDCG_at_100_ltx += res[26]
        
        ### Lime:
        res = single_user_metrics(user_vector, item_id, k, num_of_bins, num_items, rec_model, model_combined, mask_type= 'lime')    
        POS_at_1_lime += res[0]
        pos_at_5_lime += res[1]
        POS_at_10_lime += res[2]
        POS_at_20_lime += res[3]
        POS_at_50_lime += res[4]
        POS_at_100_lime += res[5]
        NEG_at_1_lime += res[6]
        NEG_at_5_lime += res[7]
        NEG_at_10_lime += res[8]
        NEG_at_20_lime += res[9]
        NEG_at_50_lime += res[10]
        NEG_at_100_lime += res[11]
        users_DEL_lime += res[12]
        users_INS_lime += res[13]
        rank_at_1_lime += res[14]
        rank_at_5_lime += res[15]
        rank_at_10_A_lime += res[16]
        rank_at_20_A_lime += res[17]
        rank_at_50_A_lime += res[18]
        rank_at_100_A_lime += res[19]
        rank_at_k_B_lime += res[20]
        NDCG_at_1_lime += res[21]
        NDCG_at_5_lime += res[22]
        NDCG_at_10_lime += res[23]
        NDCG_at_20_lime += res[24]
        NDCG_at_50_lime += res[25]
        NDCG_at_100_lime += res[26]

        if(i%100 == 0):
            print(i)
           
a = i+1

print('POS_at_1_j_u: ', np.mean(POS_at_1_j_u[1:])/a)
print('pos_at_5_j_u: ', np.mean(pos_at_5_j_u[1:])/a)
print('POS_at_10_j_u: ', np.mean(POS_at_10_j_u[1:])/a)
print('POS_at_20_j_u: ', np.mean(POS_at_20_j_u[1:])/a)
print('POS_at_50_j_u: ', np.mean(POS_at_50_j_u[1:])/a)
print('POS_at_100_j_u: ', np.mean(POS_at_100_j_u[1:])/a)
print('NEG_at_1_j_u: ', np.mean(NEG_at_1_j_u[1:])/a)
print('NEG_at_5_j_u: ', np.mean(NEG_at_5_j_u[1:])/a)
print('NEG_at_10_j_u: ', np.mean(NEG_at_10_j_u[1:])/a)
print('NEG_at_20_j_u: ', np.mean(NEG_at_20_j_u[1:])/a)
print('NEG_at_50_j_u: ', np.mean(NEG_at_50_j_u[1:])/a)
print('NEG_at_100_j_u: ', np.mean(NEG_at_100_j_u[1:])/a)
print('users_DEL_j_u: ', np.mean(users_DEL_j_u[1:])/a)
print('users_INS_j_u: ', np.mean(users_INS_j_u[1:])/a)
print('rank_at_1_j_u: ', np.mean(rank_at_1_j_u[1:])/a)
print('rank_at_5_j_u: ', np.mean(rank_at_5_j_u[1:])/a)
print('rank_at_10_A_j_u: ', np.mean(rank_at_10_A_j_u[1:])/a)
print('rank_at_20_A_j_u: ', np.mean(rank_at_20_A_j_u[1:])/a)
print('rank_at_50_A_j_u: ', np.mean(rank_at_50_A_j_u[1:])/a)
print('rank_at_100_A_j_u: ', np.mean(rank_at_100_A_j_u[1:])/a)
print('rank_at_k_B_j_u: ', np.mean(rank_at_k_B_j_u[1:])/a)
print('NDCG_at_1_j_u: ', np.mean(NDCG_at_1_j_u[1:])/a)
print('NDCG_at_5_j_u: ', np.mean(NDCG_at_5_j_u[1:])/a)
print('NDCG_at_10_j_u: ', np.mean(NDCG_at_10_j_u[1:])/a)
print('NDCG_at_20_j_u: ', np.mean(NDCG_at_20_j_u[1:])/a)
print('NDCG_at_50_j_u: ', np.mean(NDCG_at_50_j_u[1:])/a)
print('NDCG_at_100_j_u: ', np.mean(NDCG_at_100_j_u[1:])/a)

print('POS_at_1_j_g: ', np.mean(POS_at_1_j_g[1:])/a)
print('pos_at_5_j_g: ', np.mean(pos_at_5_j_g[1:])/a)
print('POS_at_10_j_g: ', np.mean(POS_at_10_j_g[1:])/a)
print('POS_at_20_j_g: ', np.mean(POS_at_20_j_g[1:])/a)
print('POS_at_50_j_g: ', np.mean(POS_at_50_j_g[1:])/a)
print('POS_at_100_j_g: ', np.mean(POS_at_100_j_g[1:])/a)
print('NEG_at_1_j_g: ', np.mean(NEG_at_1_j_g[1:])/a)
print('NEG_at_5_j_g: ', np.mean(NEG_at_5_j_g[1:])/a)
print('NEG_at_10_j_g: ', np.mean(NEG_at_10_j_g[1:])/a)
print('NEG_at_20_j_g: ', np.mean(NEG_at_20_j_g[1:])/a)
print('NEG_at_50_j_g: ', np.mean(NEG_at_50_j_g[1:])/a)
print('NEG_at_100_j_g: ', np.mean(NEG_at_100_j_g[1:])/a)
print('users_DEL_j_g: ', np.mean(users_DEL_j_g[1:])/a)
print('users_INS_j_g: ', np.mean(users_INS_j_g[1:])/a)
print('rank_at_1_j_g: ', np.mean(rank_at_1_j_g[1:])/a)
print('rank_at_5_j_g: ', np.mean(rank_at_5_j_g[1:])/a)
print('rank_at_10_A_j_g: ', np.mean(rank_at_10_A_j_g[1:])/a)
print('rank_at_20_A_j_g: ', np.mean(rank_at_20_A_j_g[1:])/a)
print('rank_at_50_A_j_g: ', np.mean(rank_at_50_A_j_g[1:])/a)
print('rank_at_100_A_j_g: ', np.mean(rank_at_100_A_j_g[1:])/a)
print('rank_at_k_B_j_g: ', np.mean(rank_at_k_B_j_g[1:])/a)
print('NDCG_at_1_j_g: ', np.mean(NDCG_at_1_j_g[1:])/a)
print('NDCG_at_5_j_g: ', np.mean(NDCG_at_5_j_g[1:])/a)
print('NDCG_at_10_j_g: ', np.mean(NDCG_at_10_j_g[1:])/a)
print('NDCG_at_20_j_g: ', np.mean(NDCG_at_20_j_g[1:])/a)
print('NDCG_at_50_j_g: ', np.mean(NDCG_at_50_j_g[1:])/a)
print('NDCG_at_100_j_g: ', np.mean(NDCG_at_100_j_g[1:])/a)

print('POS_at_1_c_s: ', np.mean(POS_at_1_c_s[1:])/a)
print('pos_at_5_c_s: ', np.mean(pos_at_5_c_s[1:])/a)
print('POS_at_10_c_s: ', np.mean(POS_at_10_c_s[1:])/a)
print('POS_at_20_c_s: ', np.mean(POS_at_20_c_s[1:])/a)
print('POS_at_50_c_s: ', np.mean(POS_at_50_c_s[1:])/a)
print('POS_at_100_c_s: ', np.mean(POS_at_100_c_s[1:])/a)
print('NEG_at_1_c_s: ', np.mean(NEG_at_1_c_s[1:])/a)
print('NEG_at_5_c_s: ', np.mean(NEG_at_5_c_s[1:])/a)
print('NEG_at_10_c_s: ', np.mean(NEG_at_10_c_s[1:])/a)
print('NEG_at_20_c_s: ', np.mean(NEG_at_20_c_s[1:])/a)
print('NEG_at_50_c_s: ', np.mean(NEG_at_50_c_s[1:])/a)
print('NEG_at_100_c_s: ', np.mean(NEG_at_100_c_s[1:])/a)
print('users_DEL_c_s: ', np.mean(users_DEL_c_s[1:])/a)
print('users_INS_c_s: ', np.mean(users_INS_c_s[1:])/a)
print('rank_at_1_c_s: ', np.mean(rank_at_1_c_s[1:])/a)
print('rank_at_5_c_s: ', np.mean(rank_at_5_c_s[1:])/a)
print('rank_at_10_A_c_s: ', np.mean(rank_at_10_A_c_s[1:])/a)
print('rank_at_20_A_c_s: ', np.mean(rank_at_20_A_c_s[1:])/a)
print('rank_at_50_A_c_s: ', np.mean(rank_at_50_A_c_s[1:])/a)
print('rank_at_100_A_c_s: ', np.mean(rank_at_100_A_c_s[1:])/a)
print('rank_at_k_B_c_s: ', np.mean(rank_at_k_B_c_s[1:])/a)
print('NDCG_at_1_c_s: ', np.mean(NDCG_at_1_c_s[1:])/a)
print('NDCG_at_5_c_s: ', np.mean(NDCG_at_5_c_s[1:])/a)
print('NDCG_at_10_c_s: ', np.mean(NDCG_at_10_c_s[1:])/a)
print('NDCG_at_20_c_s: ', np.mean(NDCG_at_20_c_s[1:])/a)
print('NDCG_at_50_c_s: ', np.mean(NDCG_at_50_c_s[1:])/a)
print('NDCG_at_100_c_s: ', np.mean(NDCG_at_100_c_s[1:])/a)

print('POS_at_1_pop: ', np.mean(POS_at_1_pop[1:])/a)
print('pos_at_5_pop: ', np.mean(pos_at_5_pop[1:])/a)
print('POS_at_10_pop: ', np.mean(POS_at_10_pop[1:])/a)
print('POS_at_20_pop: ', np.mean(POS_at_20_pop[1:])/a)
print('POS_at_50_pop: ', np.mean(POS_at_50_pop[1:])/a)
print('POS_at_100_pop: ', np.mean(POS_at_100_pop[1:])/a)
print('NEG_at_1_pop: ', np.mean(NEG_at_1_pop[1:])/a)
print('NEG_at_5_pop: ', np.mean(NEG_at_5_pop[1:])/a)
print('NEG_at_10_pop: ', np.mean(NEG_at_10_pop[1:])/a)
print('NEG_at_20_pop: ', np.mean(NEG_at_20_pop[1:])/a)
print('NEG_at_50_pop: ', np.mean(NEG_at_50_pop[1:])/a)
print('NEG_at_100_pop: ', np.mean(NEG_at_100_pop[1:])/a)
print('users_DEL_pop: ', np.mean(users_DEL_pop[1:])/a)
print('users_INS_pop: ', np.mean(users_INS_pop[1:])/a)
print('rank_at_1_pop: ', np.mean(rank_at_1_pop[1:])/a)
print('rank_at_5_pop: ', np.mean(rank_at_5_pop[1:])/a)
print('rank_at_10_A_pop: ', np.mean(rank_at_10_A_pop[1:])/a)
print('rank_at_20_A_pop: ', np.mean(rank_at_20_A_pop[1:])/a)
print('rank_at_50_A_pop: ', np.mean(rank_at_50_A_pop[1:])/a)
print('rank_at_100_A_pop: ', np.mean(rank_at_100_A_pop[1:])/a)
print('rank_at_k_B_pop: ', np.mean(rank_at_k_B_pop[1:])/a)
print('NDCG_at_1_pop: ', np.mean(NDCG_at_1_pop[1:])/a)
print('NDCG_at_5_pop: ', np.mean(NDCG_at_5_pop[1:])/a)
print('NDCG_at_10_pop: ', np.mean(NDCG_at_10_pop[1:])/a)
print('NDCG_at_20_pop: ', np.mean(NDCG_at_20_pop[1:])/a)
print('NDCG_at_50_pop: ', np.mean(NDCG_at_50_pop[1:])/a)
print('NDCG_at_100_pop: ', np.mean(NDCG_at_100_pop[1:])/a)

print('POS_at_1_lime: ', np.mean(POS_at_1_lime[1:])/a)
print('pos_at_5_lime: ', np.mean(pos_at_5_lime[1:])/a)
print('POS_at_10_lime: ', np.mean(POS_at_10_lime[1:])/a)
print('POS_at_20_lime: ', np.mean(POS_at_20_lime[1:])/a)
print('POS_at_50_lime: ', np.mean(POS_at_50_lime[1:])/a)
print('POS_at_100_lime: ', np.mean(POS_at_100_lime[1:])/a)
print('NEG_at_1_lime: ', np.mean(NEG_at_1_lime[1:])/a)
print('NEG_at_5_lime: ', np.mean(NEG_at_5_lime[1:])/a)
print('NEG_at_10_lime: ', np.mean(NEG_at_10_lime[1:])/a)
print('NEG_at_20_lime: ', np.mean(NEG_at_20_lime[1:])/a)
print('NEG_at_50_lime: ', np.mean(NEG_at_50_lime[1:])/a)
print('NEG_at_100_lime: ', np.mean(NEG_at_100_lime[1:])/a)
print('users_DEL_lime: ', np.mean(users_DEL_lime[1:])/a)
print('users_INS_lime: ', np.mean(users_INS_lime[1:])/a)
print('rank_at_1_lime: ', np.mean(rank_at_1_lime[1:])/a)
print('rank_at_5_lime: ', np.mean(rank_at_5_lime[1:])/a)
print('rank_at_10_A_lime: ', np.mean(rank_at_10_A_lime[1:])/a)
print('rank_at_20_A_lime: ', np.mean(rank_at_20_A_lime[1:])/a)
print('rank_at_50_A_lime: ', np.mean(rank_at_50_A_lime[1:])/a)
print('rank_at_100_A_lime: ', np.mean(rank_at_100_A_lime[1:])/a)
print('rank_at_k_B_lime: ', np.mean(rank_at_k_B_lime[1:])/a)
print('NDCG_at_1_lime: ', np.mean(NDCG_at_1_lime[1:])/a)
print('NDCG_at_5_lime: ', np.mean(NDCG_at_5_lime[1:])/a)
print('NDCG_at_10_lime: ', np.mean(NDCG_at_10_lime[1:])/a)
print('NDCG_at_20_lime: ', np.mean(NDCG_at_20_lime[1:])/a)
print('NDCG_at_50_lime: ', np.mean(NDCG_at_50_lime[1:])/a)
print('NDCG_at_100_lime: ', np.mean(NDCG_at_100_lime[1:])/a)

print('POS_at_1_tf_idf: ', np.mean(POS_at_1_tf_idf[1:])/a)
print('pos_at_5_tf_idf: ', np.mean(pos_at_5_tf_idf[1:])/a)
print('POS_at_10_tf_idf: ', np.mean(POS_at_10_tf_idf[1:])/a)
print('POS_at_20_tf_idf: ', np.mean(POS_at_20_tf_idf[1:])/a)
print('POS_at_50_tf_idf: ', np.mean(POS_at_50_tf_idf[1:])/a)
print('POS_at_100_tf_idf: ', np.mean(POS_at_100_tf_idf[1:])/a)
print('NEG_at_1_tf_idf: ', np.mean(NEG_at_1_tf_idf[1:])/a)
print('NEG_at_5_tf_idf: ', np.mean(NEG_at_5_tf_idf[1:])/a)
print('NEG_at_10_tf_idf: ', np.mean(NEG_at_10_tf_idf[1:])/a)
print('NEG_at_20_tf_idf: ', np.mean(NEG_at_20_tf_idf[1:])/a)
print('NEG_at_50_tf_idf: ', np.mean(NEG_at_50_tf_idf[1:])/a)
print('NEG_at_100_tf_idf: ', np.mean(NEG_at_100_tf_idf[1:])/a)
print('users_DEL_tf_idf: ', np.mean(users_DEL_tf_idf[1:])/a)
print('users_INS_tf_idf: ', np.mean(users_INS_tf_idf[1:])/a)
print('rank_at_1_tf_idf: ', np.mean(rank_at_1_tf_idf[1:])/a)
print('rank_at_5_tf_idf: ', np.mean(rank_at_5_tf_idf[1:])/a)
print('rank_at_10_A_tf_idf: ', np.mean(rank_at_10_A_tf_idf[1:])/a)
print('rank_at_20_A_tf_idf: ', np.mean(rank_at_20_A_tf_idf[1:])/a)
print('rank_at_50_A_tf_idf: ', np.mean(rank_at_50_A_tf_idf[1:])/a)
print('rank_at_100_A_tf_idf: ', np.mean(rank_at_100_A_tf_idf[1:])/a)
print('rank_at_k_B_tf_idf: ', np.mean(rank_at_k_B_tf_idf[1:])/a)
print('NDCG_at_1_tf_idf: ', np.mean(NDCG_at_1_tf_idf[1:])/a)
print('NDCG_at_5_tf_idf: ', np.mean(NDCG_at_5_tf_idf[1:])/a)
print('NDCG_at_10_tf_idf: ', np.mean(NDCG_at_10_tf_idf[1:])/a)
print('NDCG_at_20_tf_idf: ', np.mean(NDCG_at_20_tf_idf[1:])/a)
print('NDCG_at_50_tf_idf: ', np.mean(NDCG_at_50_tf_idf[1:])/a)
print('NDCG_at_100_tf_idf: ', np.mean(NDCG_at_100_tf_idf[1:])/a)

print('POS_at_1_ltx: ', np.mean(POS_at_1_ltx[1:])/a)
print('pos_at_5_ltx: ', np.mean(pos_at_5_ltx[1:])/a)
print('POS_at_10_ltx: ', np.mean(POS_at_10_ltx[1:])/a)
print('POS_at_20_ltx: ', np.mean(POS_at_20_ltx[1:])/a)
print('POS_at_50_ltx: ', np.mean(POS_at_50_ltx[1:])/a)
print('POS_at_100_ltx: ', np.mean(POS_at_100_ltx[1:])/a)
print('NEG_at_1_ltx: ', np.mean(NEG_at_1_ltx[1:])/a)
print('NEG_at_5_ltx: ', np.mean(NEG_at_5_ltx[1:])/a)
print('NEG_at_10_ltx: ', np.mean(NEG_at_10_ltx[1:])/a)
print('NEG_at_20_ltx: ', np.mean(NEG_at_20_ltx[1:])/a)
print('NEG_at_50_ltx: ', np.mean(NEG_at_50_ltx[1:])/a)
print('NEG_at_100_ltx: ', np.mean(NEG_at_100_ltx[1:])/a)
print('users_DEL_ltx: ', np.mean(users_DEL_ltx[1:])/a)
print('users_INS_ltx: ', np.mean(users_INS_ltx[1:])/a)
print('rank_at_1_ltx: ', np.mean(rank_at_1_ltx[1:])/a)
print('rank_at_5_ltx: ', np.mean(rank_at_5_ltx[1:])/a)
print('rank_at_10_A_ltx: ', np.mean(rank_at_10_A_ltx[1:])/a)
print('rank_at_20_A_ltx: ', np.mean(rank_at_20_A_ltx[1:])/a)
print('rank_at_50_A_ltx: ', np.mean(rank_at_50_A_ltx[1:])/a)
print('rank_at_100_A_ltx: ', np.mean(rank_at_100_A_ltx[1:])/a)
print('rank_at_k_B_ltx: ', np.mean(rank_at_k_B_ltx[1:])/a)
print('NDCG_at_1_ltx: ', np.mean(NDCG_at_1_ltx[1:])/a)
print('NDCG_at_5_ltx: ', np.mean(NDCG_at_5_ltx[1:])/a)
print('NDCG_at_10_ltx: ', np.mean(NDCG_at_10_ltx[1:])/a)
print('NDCG_at_20_ltx: ', np.mean(NDCG_at_20_ltx[1:])/a)
print('NDCG_at_50_ltx: ', np.mean(NDCG_at_50_ltx[1:])/a)
print('NDCG_at_100_ltx: ', np.mean(NDCG_at_100_ltx[1:])/a)

0
0
100
200
300
400
500
500
600
700
800
900
1000
1000
1100
1200
1300
1400
1500
1500


In [52]:
'''
import time
# Evaluate the model on the test set
num_of_bins = 11
k = 100

POS_at_10_ltx = np.zeros(num_of_bins)
POS_at_20_ltx = np.zeros(num_of_bins)
POS_at_50_ltx = np.zeros(num_of_bins)
POS_at_100_ltx = np.zeros(num_of_bins)
NEG_at_10_ltx = np.zeros(num_of_bins)
NEG_at_20_ltx = np.zeros(num_of_bins)
NEG_at_50_ltx = np.zeros(num_of_bins)
NEG_at_100_ltx = np.zeros(num_of_bins)
users_DEL_ltx = np.zeros(num_of_bins)
users_INS_ltx = np.zeros(num_of_bins)
rank_at_10_A_ltx = np.zeros(num_of_bins)
rank_at_20_A_ltx = np.zeros(num_of_bins)
rank_at_50_A_ltx = np.zeros(num_of_bins)
rank_at_100_A_ltx = np.zeros(num_of_bins)
rank_at_k_B_ltx = np.zeros(num_of_bins)
NDCG_at_10_ltx = np.zeros(num_of_bins)
NDCG_at_20_ltx = np.zeros(num_of_bins)
NDCG_at_50_ltx = np.zeros(num_of_bins)
NDCG_at_100_ltx = np.zeros(num_of_bins)

POS_at_10_lime= np.zeros(num_of_bins)
POS_at_20_lime = np.zeros(num_of_bins)
POS_at_50_lime = np.zeros(num_of_bins)
POS_at_100_lime = np.zeros(num_of_bins)
NEG_at_10_lime = np.zeros(num_of_bins)
NEG_at_20_lime = np.zeros(num_of_bins)
NEG_at_50_lime = np.zeros(num_of_bins)
NEG_at_100_lime = np.zeros(num_of_bins)
users_DEL_lime = np.zeros(num_of_bins)
users_INS_lime = np.zeros(num_of_bins)
rank_at_10_A_lime = np.zeros(num_of_bins)
rank_at_20_A_lime = np.zeros(num_of_bins)
rank_at_50_A_lime = np.zeros(num_of_bins)
rank_at_100_A_lime = np.zeros(num_of_bins)
rank_at_k_B_lime = np.zeros(num_of_bins)
NDCG_at_10_lime = np.zeros(num_of_bins)
NDCG_at_20_lime = np.zeros(num_of_bins)
NDCG_at_50_lime = np.zeros(num_of_bins)
NDCG_at_100_lime = np.zeros(num_of_bins)

num_of_bins = 10

explainer_model_g.eval()
model_combined.eval()

with torch.no_grad():
    for i in range(test_array.shape[0]):

        if i%500 == 0:
            print(i)
        #item_id = test_array[i][-1]
        user_id = test_array[i][-2]

        #item_vector = items_values.iloc[item_id]
        user_vector = test_array[i][:-2]

        y_pred = topk_test[user_id]
        item_id = np.argmax(y_pred)

        item_vector = items_values_dict[item_id]
        item_tensor = torch.FloatTensor(item_vector).to(device)

        user_vector[item_id] = 0
        user_tensor = torch.FloatTensor(user_vector).to(device)

        
        ### Lime:
        res = single_user_metrics(user_vector, item_id, k, num_of_bins, num_items, rec_model, model_combined, mask_type= 'lime')    
        POS_at_10_lime+= res[0]
        POS_at_20_lime += res[1]
        POS_at_50_lime += res[2]
        POS_at_100_lime += res[3]
        NEG_at_10_lime += res[4]
        NEG_at_20_lime += res[5]
        NEG_at_50_lime += res[6]
        NEG_at_100_lime += res[7]
        users_DEL_lime += res[8]
        users_INS_lime += res[9]
        rank_at_10_A_lime += res[10]
        rank_at_20_A_lime += res[11]
        rank_at_50_A_lime += res[12]
        rank_at_100_A_lime += res[13]
        rank_at_k_B_lime += res[14]
        NDCG_at_10_lime += res[15]
        NDCG_at_20_lime += res[16]
        NDCG_at_50_lime += res[17]
        NDCG_at_100_lime += res[18]
        
        ### LTX:
        res = single_user_metrics(user_vector, item_id, k, num_of_bins, num_items, rec_model, model_combined, mask_type= 'ltx')    
        POS_at_10_ltx += res[0]
        POS_at_20_ltx += res[1]
        POS_at_50_ltx += res[2]
        POS_at_100_ltx += res[3]
        NEG_at_10_ltx += res[4]
        NEG_at_20_ltx += res[5]
        NEG_at_50_ltx += res[6]
        NEG_at_100_ltx += res[7]
        users_DEL_ltx += res[8]
        users_INS_ltx += res[9]
        rank_at_10_A_ltx += res[10]
        rank_at_20_A_ltx += res[11]
        rank_at_50_A_ltx += res[12]
        rank_at_100_A_ltx += res[13]
        rank_at_k_B_ltx += res[14]
        NDCG_at_10_ltx += res[15]
        NDCG_at_20_ltx += res[16]
        NDCG_at_50_ltx += res[17]
        NDCG_at_100_ltx += res[18]

        if(i%100 == 0):
            print(i)
           
a = i+1


print("POS_at_10_lime", np.mean(POS_at_10_lime[1:])/a)
print("POS_at_20_lime", np.mean(POS_at_20_lime[1:])/a)
print("POS_at_50_lime", np.mean(POS_at_50_lime[1:])/a)
print("POS_at_100_lime", np.mean(POS_at_100_lime[1:])/a)
print("NEG_at_10_lime", np.mean(NEG_at_10_lime[1:])/a)
print("NEG_at_20_lime", np.mean(NEG_at_20_lime[1:])/a)
print("NEG_at_50_lime", np.mean(NEG_at_50_lime[1:])/a)
print("NEG_at_100_lime", np.mean(NEG_at_100_lime[1:])/a)
print("users_DEL_lime", np.mean(users_DEL_lime[1:])/a)
print("users_INS_lime", np.mean(users_INS_lime[1:])/a)
print("rank_at_10_A_lime", np.mean(rank_at_10_A_lime[1:])/a)
print("rank_at_20_A_lime", np.mean(rank_at_20_A_lime[1:])/a)
print("rank_at_50_A_lime", np.mean(rank_at_50_A_lime[1:])/a)
print("rank_at_100_A_lime", np.mean(rank_at_100_A_lime[1:])/a)
print("rank_at_k_B_lime", np.mean(rank_at_k_B_lime[1:])/a)
print('NDCG_at_10_lime', np.mean(NDCG_at_10_lime[1:])/a)
print('NDCG_at_20_lime', np.mean(NDCG_at_20_lime[1:])/a)
print('NDCG_at_50_lime', np.mean(NDCG_at_50_lime[1:])/a)
print('NDCG_at_100_lime', np.mean(NDCG_at_100_lime[1:])/a)

print("POS_at_10_ltx", np.mean(POS_at_10_ltx[1:])/a)
print("POS_at_20_ltx", np.mean(POS_at_20_ltx[1:])/a)
print("POS_at_50_ltx", np.mean(POS_at_50_ltx[1:])/a)
print("POS_at_100_ltx", np.mean(POS_at_100_ltx[1:])/a)
print("NEG_at_10_ltx", np.mean(NEG_at_10_ltx[1:])/a)
print("NEG_at_20_ltx", np.mean(NEG_at_20_ltx[1:])/a)
print("NEG_at_50_ltx", np.mean(NEG_at_50_ltx[1:])/a)
print("NEG_at_100_ltx", np.mean(NEG_at_100_ltx[1:])/a)
print("users_DEL_ltx", np.mean(users_DEL_ltx[1:])/a)
print("users_INS_ltx", np.mean(users_INS_ltx[1:])/a)
print("rank_at_10_A_ltx", np.mean(rank_at_10_A_ltx[1:])/a)
print("rank_at_20_A_ltx", np.mean(rank_at_20_A_ltx[1:])/a)
print("rank_at_50_A_ltx", np.mean(rank_at_50_A_ltx[1:])/a)
print("rank_at_100_A_ltx", np.mean(rank_at_100_A_ltx[1:])/a)
print("rank_at_k_B_ltx", np.mean(rank_at_k_B_ltx[1:])/a)
print('NDCG_at_10_ltx', np.mean(NDCG_at_10_ltx[1:])/a)
print('NDCG_at_20_ltx', np.mean(NDCG_at_20_ltx[1:])/a)
print('NDCG_at_50_ltx', np.mean(NDCG_at_50_ltx[1:])/a)
print('NDCG_at_100_ltx', np.mean(NDCG_at_100_ltx[1:])/a)
'''

0
0
100
200
300
400
500
500
600
700
800
900
1000
1000
1100
1200
1300
1400
1500
1500
1600
1700
1800
1900
2000
2000
2100
2200
2300
2400
2500
2500
2600
2700
POS_at_10_lime 0.17894353369763205
POS_at_20_lime 0.225063752276867
POS_at_50_lime 0.3081967213114754
POS_at_100_lime 0.3943169398907104
NEG_at_10_lime 0.5574134790528232
NEG_at_20_lime 0.6199635701275046
NEG_at_50_lime 0.7037158469945355
NEG_at_100_lime 0.771912568306011
users_DEL_lime 0.7144514225977017
users_INS_lime 0.9868090570800286
rank_at_10_A_lime 0.1411001821493624
rank_at_20_A_lime 0.172874316939891
rank_at_50_A_lime 0.23223096539162155
rank_at_100_A_lime 0.2944571948998188
rank_at_k_B_lime 0.11915433067784759
NDCG_at_10_lime 0.12694719490132045
NDCG_at_20_lime 0.13848198213972462
NDCG_at_50_lime 0.1548899496117359
NDCG_at_100_lime 0.16885818025670632
POS_at_10_ltx 0.20127504553734063
POS_at_20_ltx 0.25249544626593806
POS_at_50_ltx 0.35267759562841533
POS_at_100_ltx 0.447103825136612
NEG_at_10_ltx 0.5377777777777778
NEG_at_

In [53]:
names = ['POS_at_10_j_u', 'POS_at_20_j_u', 'POS_at_50_j_u', 'POS_at_100_j_u', 'NEG_at_10_j_u', 'NEG_at_20_j_u', 'NEG_at_50_j_u', 'NEG_at_100_j_u', 'users_DEL_j_u', 'users_INS_j_u', 'rank_at_10_A_j_u', 'rank_at_20_A_j_u', 'rank_at_50_A_j_u', 'rank_at_100_A_j_u', 'rank_at_k_B_j_u', 'NDCG_at_10_j_u', 'NDCG_at_20_j_u', 'NDCG_at_50_j_u', 'NDCG_at_100_j_u', 'POS_at_10_j_g', 'POS_at_20_j_g', 'POS_at_50_j_g', 'POS_at_100_j_g', 'NEG_at_10_j_g', 'NEG_at_20_j_g', 'NEG_at_50_j_g', 'NEG_at_100_j_g', 'users_DEL_j_g', 'users_INS_j_g', 'rank_at_10_A_j_g', 'rank_at_20_A_j_g', 'rank_at_50_A_j_g', 'rank_at_100_A_j_g', 'rank_at_k_B_j_g', 'NDCG_at_10_j_g', 'NDCG_at_20_j_g', 'NDCG_at_50_j_g', 'NDCG_at_100_j_g', 'POS_at_10_c_s', 'POS_at_20_c_s', 'POS_at_50_c_s', 'POS_at_100_c_s', 'NEG_at_10_c_s', 'NEG_at_20_c_s', 'NEG_at_50_c_s', 'NEG_at_100_c_s', 'users_DEL_c_s', 'users_INS_c_s', 'rank_at_10_A_c_s', 'rank_at_20_A_c_s', 'rank_at_50_A_c_s', 'rank_at_100_A_c_s', 'rank_at_k_B_c_s', 'NDCG_at_10_c_s', 'NDCG_at_20_c_s', 'NDCG_at_50_c_s', 'NDCG_at_100_c_s', 'POS_at_10_pop', 'POS_at_20_pop', 'POS_at_50_pop', 'POS_at_100_pop', 'NEG_at_10_pop', 'NEG_at_20_pop', 'NEG_at_50_pop', 'NEG_at_100_pop', 'users_DEL_pop', 'users_INS_pop', 'rank_at_10_A_pop', 'rank_at_20_A_pop', 'rank_at_50_A_pop', 'rank_at_100_A_pop', 'rank_at_k_B_pop', 'NDCG_at_10_pop', 'NDCG_at_20_pop', 'NDCG_at_50_pop', 'NDCG_at_100_pop', 'POS_at_10_lime', 'POS_at_20_lime', 'POS_at_50_lime', 'POS_at_100_lime', 'NEG_at_10_lime', 'NEG_at_20_lime', 'NEG_at_50_lime', 'NEG_at_100_lime', 'users_DEL_lime', 'users_INS_lime', 'rank_at_10_A_lime', 'rank_at_20_A_lime', 'rank_at_50_A_lime', 'rank_at_100_A_lime', 'rank_at_k_B_lime', 'NDCG_at_10_lime', 'NDCG_at_20_lime', 'NDCG_at_50_lime', 'NDCG_at_100_lime', 'POS_at_10_ltx', 'POS_at_20_ltx', 'POS_at_50_ltx', 'POS_at_100_ltx', 'NEG_at_10_ltx', 'NEG_at_20_ltx', 'NEG_at_50_ltx', 'NEG_at_100_ltx', 'users_DEL_ltx', 'users_INS_ltx', 'rank_at_10_A_ltx', 'rank_at_20_A_ltx', 'rank_at_50_A_ltx', 'rank_at_100_A_ltx', 'rank_at_k_B_ltx', 'NDCG_at_10_ltx', 'NDCG_at_20_ltx', 'NDCG_at_50_ltx', 'NDCG_at_100_ltx', 'POS_at_10_tf', 'POS_at_20_tf', 'POS_at_50_tf', 'POS_at_100_tf', 'NEG_at_10_tf', 'NEG_at_20_tf', 'NEG_at_50_tf', 'NEG_at_100_tf', 'users_DEL_tf', 'users_INS_tf', 'rank_at_10_A_tf', 'rank_at_20_A_tf', 'rank_at_50_A_tf', 'rank_at_100_A_tf', 'rank_at_k_B_tf', 'NDCG_at_10_tf', 'NDCG_at_20_tf', 'NDCG_at_50_tf', 'NDCG_at_100_tf']

values = [POS_at_10_j_u, POS_at_20_j_u, POS_at_50_j_u, POS_at_100_j_u, NEG_at_10_j_u, NEG_at_20_j_u, NEG_at_50_j_u, NEG_at_100_j_u, users_DEL_j_u, users_INS_j_u,rank_at_10_A_j_u,rank_at_20_A_j_u, rank_at_50_A_j_u, rank_at_100_A_j_u, rank_at_k_B_j_u, NDCG_at_10_j_u, NDCG_at_20_j_u, NDCG_at_50_j_u, NDCG_at_100_j_u, POS_at_10_j_g, POS_at_20_j_g, POS_at_50_j_g, POS_at_100_j_g, NEG_at_10_j_g, NEG_at_20_j_g, NEG_at_50_j_g, NEG_at_100_j_g, users_DEL_j_g, users_INS_j_g, rank_at_10_A_j_g, rank_at_20_A_j_g, rank_at_50_A_j_g, rank_at_100_A_j_g, rank_at_k_B_j_g, NDCG_at_10_j_g, NDCG_at_20_j_g, NDCG_at_50_j_g, NDCG_at_100_j_g, POS_at_10_c_s, POS_at_20_c_s, POS_at_50_c_s, POS_at_100_c_s, NEG_at_10_c_s, NEG_at_20_c_s, NEG_at_50_c_s, NEG_at_100_c_s, users_DEL_c_s, users_INS_c_s, rank_at_10_A_c_s, rank_at_20_A_c_s, rank_at_50_A_c_s, rank_at_100_A_c_s, rank_at_k_B_c_s, NDCG_at_10_c_s, NDCG_at_20_c_s, NDCG_at_50_c_s, NDCG_at_100_c_s, POS_at_10_pop, POS_at_20_pop, POS_at_50_pop, POS_at_100_pop, NEG_at_10_pop, NEG_at_20_pop, NEG_at_50_pop, NEG_at_100_pop, users_DEL_pop, users_INS_pop, rank_at_10_A_pop, rank_at_20_A_pop, rank_at_50_A_pop, rank_at_100_A_pop, rank_at_k_B_pop, NDCG_at_10_pop, NDCG_at_20_pop, NDCG_at_50_pop, NDCG_at_100_pop, POS_at_10_lime, POS_at_20_lime, POS_at_50_lime, POS_at_100_lime, NEG_at_10_lime, NEG_at_20_lime, NEG_at_50_lime, NEG_at_100_lime, users_DEL_lime, users_INS_lime,rank_at_10_A_lime,rank_at_20_A_lime, rank_at_50_A_lime, rank_at_100_A_lime, rank_at_k_B_lime, NDCG_at_10_lime, NDCG_at_20_lime, NDCG_at_50_lime, NDCG_at_100_lime, POS_at_10_ltx, POS_at_20_ltx, POS_at_50_ltx, POS_at_100_ltx, NEG_at_10_ltx, NEG_at_20_ltx, NEG_at_50_ltx, NEG_at_100_ltx, users_DEL_ltx, users_INS_ltx, rank_at_10_A_ltx, rank_at_20_A_ltx, rank_at_50_A_ltx, rank_at_100_A_ltx, rank_at_k_B_ltx, NDCG_at_10_ltx, NDCG_at_20_ltx, NDCG_at_50_ltx, NDCG_at_100_ltx,POS_at_10_tf, POS_at_20_tf, POS_at_50_tf, POS_at_100_tf, NEG_at_10_tf, NEG_at_20_tf, NEG_at_50_tf, NEG_at_100_tf, users_DEL_tf, users_INS_tf, rank_at_10_A_tf, rank_at_20_A_tf, rank_at_50_A_tf, rank_at_100_A_tf, rank_at_k_B_tf, NDCG_at_10_tf, NDCG_at_20_tf, NDCG_at_50_tf, NDCG_at_100_tf]

#names = ['POS_at_10_lime', 'POS_at_20_lime', 'POS_at_50_lime', 'POS_at_100_lime', 'NEG_at_10_lime', 'NEG_at_20_lime', 'NEG_at_50_lime', 'NEG_at_100_lime', 'users_DEL_lime', 'users_INS_lime', 'rank_at_10_A_lime', 'rank_at_20_A_lime', 'rank_at_50_A_lime', 'rank_at_100_A_lime', 'rank_at_k_B_lime', 'NDCG_at_10_lime', 'NDCG_at_20_lime', 'NDCG_at_50_lime', 'NDCG_at_100_lime']

#values = [POS_at_10_lime, POS_at_20_lime, POS_at_50_lime, POS_at_100_lime, NEG_at_10_lime, NEG_at_20_lime, NEG_at_50_lime, NEG_at_100_lime, users_DEL_lime, users_INS_lime,rank_at_10_A_lime,rank_at_20_A_lime, rank_at_50_A_lime, rank_at_100_A_lime, rank_at_k_B_lime, NDCG_at_10_lime, NDCG_at_20_lime, NDCG_at_50_lime, NDCG_at_100_lime]

d = {'names':names, 'values': values}

MLP_results_ml1m = pd.DataFrame(d)


normalized_values = []
for i in range(len(values)):
    lst = []
    for j in range(len(values[i])):
        lst.append(values[i][j]/a)
    normalized_values.append(lst)

MLP_results_ml1m['normalized_values'] = normalized_values

AUC = []
for i in range(len(values)):
    sublist = MLP_results_ml1m['normalized_values'][i][1:]
    AUC.append(np.mean(sublist))

MLP_results_ml1m['AUC'] = AUC

MLP_results_ml1m.head()

MLP_results_ml1m.to_csv('MLP_18_4_results_yahoo_lime_50.csv')

In [ ]:
!pip freeze > requirements.txt

In [ ]:
##Optuna

In [ ]:
def calculate_pos_top_k(user_vector, item_id, item_tensor, num_of_bins, num_items, rec_model,model_combined, k=20,y_pred=None):
    

    user_tensor = torch.FloatTensor(user_vector).to(device)
    
    POS_masked = user_tensor
    POS_masked[item_id] = 0
    user_hist_size = np.sum(user_vector) - 1
    item_vector = items_values_dict[item_id]

    bins = [0] + [len(x) for x in np.array_split(np.arange(user_hist_size), num_of_bins, axis=0)]

    POS_at_20 = [0] * (len(bins))
    total_items = 0

    
    for i in range(len(bins)):
        total_items += bins[i]
        
        if i == 0:
            sim_items = find_LTX_mask(POS_masked, item_vector, item_id,model_combined)   
             
        
        POS_sim_items  = list(sorted(sim_items.items(), key=lambda item: item[1],reverse=True))[0:user_hist_size]
     
        POS_masked = torch.zeros_like(user_tensor, dtype=torch.float32, device=device)
        for j in POS_sim_items[:total_items]:
            POS_masked[j[0]] = 1
        POS_masked = user_tensor - POS_masked

        POS_index = get_index_in_the_list(POS_masked,user_tensor, item_id, num_items, rec_model)+1
                               
        # for pos:
        POS_at_20[i] = 1 if POS_index <=20 else 0
       
        

    res = np.array(POS_at_20)
    return res

In [ ]:
import optuna

def my_objective(trial):
    # Define hyperparameters to optimize
    lr = trial.suggest_float('learning_rate', 0.001, 0.05)
    #alpha_parameter = trial.suggest_float('alpha_parameter', 0.5,1.5)
    alpha_parameter = trial.suggest_categorical('alpha_parameter', [0.1, 0.5, 1, 5, 10, 20])
    #hidden_dim = trial.suggest_categorical('hidden_dim', [20, 64, 128])
    epochs = trial.suggest_categorical('epochs', [5,10,20,30])

    train_losses = []
    hidden_dim = 20
    num_of_bins = 10
    num_of_rand_users = 50
    POS_at_20_ltx = np.zeros(num_of_bins)

    #Randomly sampled num_of_rand_users users from test array
    #random_rows = np.random.choice(test_array.shape[0], num_of_rand_users, replace=False)
    random_sampled_array = test_array[:500]

    rec_model = Recommender_G(num_items, hidden_dim)
    rec_model.load_state_dict(torch.load('recommender_model_yahoo.pt'))
    for param in rec_model.parameters():
        param.requires_grad= False

    explainer_model_g = Explainer_G(rec_model, num_items, hidden_dim).to(device) 
    model_combined = LossModelCombined(alpha_parameter, rec_model, explainer_model_g, hidden_dim).to(device)
    optimizer_comb = torch.optim.Adam(model_combined.parameters(), lr)
        
    #Train LTX
    for epoch in range(epochs):
        train_loss = 0
        model_combined.train()
        explainer_model_g.train()
        for i in range(train_unique_arr.shape[0]):
                
            #user data
            user_id = train_unique_arr[i][-3]
            user_vector = train_unique_arr[i][:-3]
            #get top1 of this user for LTX training
                
            top1_item = np.argmax(topk_train[user_id])

            item_vector = items_values_dict[top1_item]
            item_tensor = torch.FloatTensor(item_vector).to(device)
                
            user_vector[top1_item] = 0 
            user_tensor = torch.FloatTensor(user_vector).to(device)
                
            #train model
            optimizer_comb.zero_grad()
            x_masked_g, loss_comb_g = model_combined(user_tensor, item_tensor)
                
            train_loss+=loss_comb_g.item()
                    
            loss_comb_g.backward()
            optimizer_comb.step()
          
        train_losses.append(train_loss/train_unique_arr.shape[0])
      
        #print(f"Epoch {epoch}, Train Loss {train_loss/train_unique_arr.shape[0]:.4f}")
            
    #Monitoring on POS metric after each epoch
    model_combined.eval()
    explainer_model_g.eval()
    for j in range(random_sampled_array.shape[0]):
                
        user_id = random_sampled_array[j][-2]
        user_vector = random_sampled_array[j][:-2]
        user_tensor = torch.FloatTensor(user_vector).to(device)

        #get top1 of this test user item
        top1_item_test = np.argmax(topk_test[user_id])
                
        item_vector = items_values_dict[top1_item_test]
        item_tensor = torch.FloatTensor(item_vector).to(device)
                
        user_vector[top1_item_test] = 0 
        user_tensor = torch.FloatTensor(user_vector).to(device)
                
        res = calculate_pos_top_k(user_vector, top1_item_test, item_tensor, num_of_bins, num_items, rec_model, model_combined, k=20,y_pred=None)

        POS_at_20_ltx += res       
    #print("POS@20 at epoch {:d} is {:.4f} ".format(int(epoch), np.mean(POS_at_20_ltx)/(j+1)))
    return np.mean(POS_at_20_ltx)/j 

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(my_objective, n_trials=50)

# Print best hyperparameters and corresponding metric value
print("Best hyperparameters: {}".format(study.best_params))
print("Best metric value: {}".format(study.best_value))